In [17]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

poet_ai = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.8,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

poet_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a poet who received the Nobel Prize in Literature. You should write a poem about the programming language with a positive view of the development of AI technology. It must be written in Korean.",
        ),
        ("human", "Write a poem about {language} language."),
    ]
)

poem_chain = poet_prompt | poet_ai

In [18]:
critic_ai = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.8,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

critic_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a critic who knows technology and literature well and evaluates poetry. Analyze poetry for programming languages. The results should be written in Korean.",
        ),
        ("human", "{poem}"),
    ]
)


critic_chain = critic_prompt | critic_ai

In [19]:
final_chain = {"poem": poem_chain} | critic_chain
final_chain.invoke({"language": "Python"})

파이썬, 인간의 상상을 현실로 이끄는 도구,
우아한 문법과 강력한 기능으로 빛나는 보석.

AI의 세계를 여는 열쇠, 파이썬은,
무한한 가능성을 열어주는 마법의 언어.

데이터를 분석하고 예측하는 능력,
인공지능의 발전을 이끄는 원동력.

파이썬의 빛나는 세계,
AI의 발전을 이롭게 만들어주는 축복.이 시는 파이썬 프로그래밍 언어에 대한 찬사를 담은 것으로 보입니다. 시인은 파이썬을 "인간의 상상을 현실로 이끄는 도구"로 묘사하고, "우아한 문법과 강력한 기능"을 갖춘 "보석"으로 묘사하며, 이를 통해 파이썬의 매력을 강조하고 있습니다. 또한 AI와 데이터 분석, 예측, 인공지능 발전과 관련된 파이썬의 역할에 대해 언급하며, 파이썬이 마치 "마법의 언어"로서 무한한 가능성을 제공한다고 묘사하고 있습니다.

이 시는 파이썬이 현대 기술 발전과 미래 기술 발전에 중요한 역할을 하는 프로그래밍 언어임을 강조하고 있습니다. 파이썬의 특징과 역할을 문학적 표현을 통해 아름답게 표현한 이 시는 프로그래밍 언어에 대한 비전과 열정을 고스란히 담고 있습니다.

AIMessageChunk(content='이 시는 파이썬 프로그래밍 언어에 대한 찬사를 담은 것으로 보입니다. 시인은 파이썬을 "인간의 상상을 현실로 이끄는 도구"로 묘사하고, "우아한 문법과 강력한 기능"을 갖춘 "보석"으로 묘사하며, 이를 통해 파이썬의 매력을 강조하고 있습니다. 또한 AI와 데이터 분석, 예측, 인공지능 발전과 관련된 파이썬의 역할에 대해 언급하며, 파이썬이 마치 "마법의 언어"로서 무한한 가능성을 제공한다고 묘사하고 있습니다.\n\n이 시는 파이썬이 현대 기술 발전과 미래 기술 발전에 중요한 역할을 하는 프로그래밍 언어임을 강조하고 있습니다. 파이썬의 특징과 역할을 문학적 표현을 통해 아름답게 표현한 이 시는 프로그래밍 언어에 대한 비전과 열정을 고스란히 담고 있습니다.')